In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
sparSession = SparkSession.builder.appName("MarketSales Spark Analysis").getOrCreate()

In [0]:
rawDF = sparSession.read.option("delimiter",";").csv("/FileStore/tables/MarketSales_20220215-2.csv",header=True, inferSchema=True).drop("SALESMAN","CLİENTNAME")

In [0]:
rawDF.describe().show()

+-------+--------------------+--------------------+--------------+------------------+-----------------+-----------------+------------------+-------------+------+----------+------------------+------------------+----------+--------------+---------------+-----------------+--------------+--------------+------+
|summary|                  ID|            ITEMNAME|         DATE_|            AMOUNT|            PRICE|     LINENETTOTAL|          BRANCHNR|       BRANCH|  CITY|    REGION|        CLIENTCODE|         BRANDCODE|     BRAND|CATEGORY_NAME1| CATEGORY_NAME2|   CATEGORY_NAME3|     STARTDATE|       ENDDATE|GENDER|
+-------+--------------------+--------------------+--------------+------------------+-----------------+-----------------+------------------+-------------+------+----------+------------------+------------------+----------+--------------+---------------+-----------------+--------------+--------------+------+
|  count|              611115|              603941|        611099|          

In [0]:
rawDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- LINENETTOTAL: double (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (nullable = true)
 |-- ENDDATE: string (nullable = true)
 |-- GENDER: string (nullable = true)



In [0]:
selectrawDF = rawDF.select("ITEMNAME","AMOUNT","PRICE","LINENETTOTAL","BRANCH")

In [0]:
selectrawDF.show()

+--------------------+------+-----+------------+----------------+
|            ITEMNAME|AMOUNT|PRICE|LINENETTOTAL|          BRANCH|
+--------------------+------+-----+------------+----------------+
|SPRITE 1 LT LIMON...|   1.0|  2.0|         2.0|  Kocaeli Subesi|
|           TOZ SEKER|   5.0| 2.65|       13.25|  Antalya Subesi|
|FALIM SAKIZ 5LI NANE|   1.0|  0.4|         0.4| �stanbul Subesi|
|FALIM SAKIZ 5LI NANE|   1.0|  0.4|         0.4| �stanbul Subesi|
|FALIM SAKIZ 5LI C...|   1.0|  0.4|         0.4|    �zmir Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3|   Elaz�� Subesi|
|F NEFFIS TOZ SEKE...|   1.0|  5.6|         5.6|  Malatya Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3| �stanbul Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3|   Manisa Subesi|
|F NEFFIS KESME SE...|   1.0| 2.85|        2.85|    Mu�la Subesi|
|KENT TOPITOP DISN...|   1.0|  1.6|         1.6|      Mu� Subesi|
|VIVIDENT CZD FRUI...|   1.0| 1.95|        1.95|    Adana Subesi|
|LAVACHE Q

In [0]:
selectrawDF.agg({"LINENETTOTAL":"sum"}).show()

+------------------+
| sum(LINENETTOTAL)|
+------------------+
|2478409.4400002426|
+------------------+



In [0]:
sumTotalByBranchDF = selectrawDF.groupBy("BRANCH").sum("LINENETTOTAL").orderBy("sum(LINENETTOTAL)", ascending=False)

In [0]:
rDF = sumTotalByBranchDF.withColumn("totalAmount",round("sum(LINENETTOTAL)",)).drop("sum(LINENETTOTAL)")

In [0]:
display(rDF)

BRANCH,totalAmount
�stanbul Subesi,458485.0
Ankara Subesi,167002.0
�zmir Subesi,131799.0
Bursa Subesi,81149.0
Antalya Subesi,75283.0
Adana Subesi,64981.0
Konya Subesi,63918.0
�anl�urfa Subesi,58236.0
Kocaeli Subesi,57709.0
Zonguldak Subesi,57571.0


In [0]:
selectrawDF.groupBy("ITEMNAME").count().orderBy("count", ascending=False).show(truncate=False)

+-----------------------------+-----+
|ITEMNAME                     |count|
+-----------------------------+-----+
|EKMEK 250 GR                 |9122 |
|null                         |7175 |
|DOMATES                      |6018 |
|PATATES                      |5718 |
|PORTAKAL                     |5611 |
|TOZ SEKER                    |5242 |
|SOGAN                        |4631 |
|OSMANCIK PIRINC KG.          |4101 |
|LIMON                        |4093 |
|MUZ                          |3934 |
|KIVIRCIK                     |3925 |
|MAYDANOZ                     |3808 |
|HAVUC                        |3632 |
|S�H�RL� ELLER C�G K�FTE 200GR|3566 |
|F NEFFIS S�T YARIM YA�LI 1 LT|3253 |
|YERLI BADEM                  |2900 |
|ULKER CIKOLATALI GOFRET 35GR |2692 |
|YAHYAOGLU 15 LI YUMURTA      |2588 |
|PORTAKAL SIKMALIK            |2561 |
|SALKIM DOMATES               |2471 |
+-----------------------------+-----+
only showing top 20 rows



In [0]:
rawDF.select("ITEMNAME","LINENETTOTAL","CLIENTCODE").filter("LINENETTOTAL>1000").show()

+--------------------+------------+----------+
|            ITEMNAME|LINENETTOTAL|CLIENTCODE|
+--------------------+------------+----------+
|S�H�RL� ELLER C�G...|      1439.2|    808718|
|S�H�RL� ELLER C�G...|      1799.0|      null|
|S�H�RL� ELLER C�G...|      1799.0|      null|
|                null|      1304.0|       933|
|                null|      1304.0|       933|
|     SAMSUN 216 SOFT|      2761.0|      9010|
|     SAMSUN 216 SOFT|      2761.0|      9010|
|S�H�RL� ELLER C�G...|      1078.0|    237685|
|S�H�RL� ELLER C�G...|      1078.0|    237685|
|CANPED POLITENLI ...|      4278.0|    854889|
|CANPED POLITENLI ...|      4278.0|    854889|
+--------------------+------------+----------+



In [0]:
rawDF.select("ITEMNAME","LINENETTOTAL","CLIENTCODE").filter("LINENETTOTAL>1500 and LINENETTOTAL<5000 ").show()

+--------------------+------------+----------+
|            ITEMNAME|LINENETTOTAL|CLIENTCODE|
+--------------------+------------+----------+
|S�H�RL� ELLER C�G...|      1799.0|      null|
|S�H�RL� ELLER C�G...|      1799.0|      null|
|     SAMSUN 216 SOFT|      2761.0|      9010|
|     SAMSUN 216 SOFT|      2761.0|      9010|
|CANPED POLITENLI ...|      4278.0|    854889|
|CANPED POLITENLI ...|      4278.0|    854889|
+--------------------+------------+----------+

